In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import os

os.makedirs("../outputs/aggregated_results", exist_ok=True)


# add inference results to test set
def test_set_add_inference_results(df_test_set, df_inference_results):
    df_merged = pd.merge(df_test_set, df_inference_results[['blob_path', 'hypothesis', 'processing_time_sec']], left_on='blob_path', right_on='blob_path', how='left')
    stt_tag = df_inference_results['model_name'].iloc[0]
    df_merged = df_merged.rename(columns={'hypothesis': f'inference_{stt_tag}', 'processing_time_sec': f'processing_time_{stt_tag}'})
    return df_merged

In [ ]:
df_pre2010_test_set = pd.read_parquet("../data/raw/loc/veterans_history_project_resources_pre2010_test.parquet")
# df_full = pd.read_parquet("../data/raw/loc/veterans_history_project_resources_with_audio_features.parquet")
# df_pre2010_test_set = df_full
df_pre2010_test_set.head()

In [ ]:
df_pre2010_test_set.columns

In [ ]:
# example output dfs after inference
# results from AWS Transcribe
aws_transcribe_results_filepath = "../outputs/vhp-pre2010-aws-sample500-gpu/inference_results.parquet"
df_aws_transcribe_results = pd.read_parquet(aws_transcribe_results_filepath)
df_aws_transcribe_results = df_aws_transcribe_results[df_aws_transcribe_results['error_message'].isnull()]
df_aws_transcribe_results = df_aws_transcribe_results[df_aws_transcribe_results['status']=="success"]
df_aws_transcribe_results = df_aws_transcribe_results.replace('', np.nan).dropna(subset=['hypothesis'])
print(df_aws_transcribe_results.info())
df_aws_transcribe_results.head()

In [ ]:
# results from Chirp2 (GCP)
chirp2_results_filepath = "../outputs/vhp-pre2010-chirp2-sample500-gpu/inference_results.parquet"
df_chirp2_results = pd.read_parquet(chirp2_results_filepath)
df_chirp2_results = df_chirp2_results[df_chirp2_results['error_message'].isnull()]
df_chirp2_results = df_chirp2_results[df_chirp2_results['status']=="success"]
df_chirp2_results = df_chirp2_results.replace('', np.nan).dropna(subset=['hypothesis'])
print(df_chirp2_results.info())
df_chirp2_results.head()

In [ ]:
# results from Chirp3 (GCP)
chirp3_results_filepath = "../outputs/vhp-pre2010-chirp3-sample500-gpu/inference_results.parquet"
df_chirp3_results = pd.read_parquet(chirp3_results_filepath)
df_chirp3_results = df_chirp3_results[df_chirp3_results['error_message'].isnull()]
df_chirp3_results = df_chirp3_results[df_chirp3_results['status']=="success"]
df_chirp3_results = df_chirp3_results.replace('', np.nan).dropna(subset=['hypothesis'])
print(df_chirp3_results.info())
df_chirp3_results.head()

In [ ]:
df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set, df_aws_transcribe_results)
df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set_expanded, df_chirp2_results)
df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set_expanded, df_chirp3_results)
df_pre2010_test_set_expanded = df_pre2010_test_set_expanded.dropna(subset=['inference_aws-transcribe', 'inference_gcp-chirp_2', 'inference_gcp-chirp_3'])
df_pre2010_test_set_expanded[['transcript_raw_text_only', 'inference_aws-transcribe', 'inference_gcp-chirp_2', 'inference_gcp-chirp_3']].head()

In [ ]:
len(df_pre2010_test_set_expanded)

In [ ]:
df_pre2010_test_set_expanded.to_csv("../outputs/aggregated_results/vhp-pre2010-stt-comparison-close-service.csv", index=False)

Open models

In [ ]:
# results from NVIDIA Parakeet-TDT-0.6B-v3
parakeet_results_filepath = "../outputs/vhp-pre2010-parakeet-sample1000-runpod-a6000/inference_results.parquet"
df_parakeet_results = pd.read_parquet(parakeet_results_filepath)
df_parakeet_results = df_parakeet_results[df_parakeet_results['error_message'].isnull()]
df_parakeet_results = df_parakeet_results[df_parakeet_results['status']=="success"]
df_parakeet_results = df_parakeet_results.replace('', np.nan).dropna(subset=['hypothesis'])
print(df_parakeet_results.info())
df_parakeet_results.head()

In [ ]:
# results from faster-whisper-small (distil - en)
faster_whisper_small_results_filepath = "../outputs/vhp-distil-small-en-sample1000-full-gpu/inference_results.parquet"
df_faster_whisper_small_results = pd.read_parquet(faster_whisper_small_results_filepath)
df_faster_whisper_small_results = df_faster_whisper_small_results[df_faster_whisper_small_results['error_message'].isnull()]
df_faster_whisper_small_results = df_faster_whisper_small_results[df_faster_whisper_small_results['status']=="success"]
df_faster_whisper_small_results = df_faster_whisper_small_results.replace('', np.nan).dropna(subset=['hypothesis'])
df_faster_whisper_small_results['model_name'] = 'faster-whisper-small'
print(df_faster_whisper_small_results.info())
df_faster_whisper_small_results.head()

In [ ]:
# results from faster-whisper-medium (distil - en)
faster_whisper_medium_results_filepath = "../outputs/vhp-distil-medium-en-sample1000-full-gpu/inference_results.parquet"
df_faster_whisper_medium_results = pd.read_parquet(faster_whisper_medium_results_filepath)
df_faster_whisper_medium_results = df_faster_whisper_medium_results[df_faster_whisper_medium_results['error_message'].isnull()]
df_faster_whisper_medium_results = df_faster_whisper_medium_results[df_faster_whisper_medium_results['status']=="success"]
df_faster_whisper_medium_results = df_faster_whisper_medium_results.replace('', np.nan).dropna(subset=['hypothesis'])
df_faster_whisper_medium_results['model_name'] = 'faster-whisper-medium'
print(df_faster_whisper_medium_results.info())
df_faster_whisper_medium_results.head()


In [ ]:
# results from faster-whisper-large-v3 (distil)
faster_whisper_large_v3_results_filepath = "../outputs/vhp-pre2010-distil-large-v3-full-gpu/inference_results.parquet"
df_faster_whisper_large_v3_results = pd.read_parquet(faster_whisper_large_v3_results_filepath)
df_faster_whisper_large_v3_results = df_faster_whisper_large_v3_results[df_faster_whisper_large_v3_results['error_message'].isnull()]
df_faster_whisper_large_v3_results = df_faster_whisper_large_v3_results[df_faster_whisper_large_v3_results['status']=="success"]
df_faster_whisper_large_v3_results = df_faster_whisper_large_v3_results.replace('', np.nan).dropna(subset=['hypothesis'])
df_faster_whisper_large_v3_results['model_name'] = 'faster-whisper-large-v3'
print(df_faster_whisper_large_v3_results.info())
df_faster_whisper_large_v3_results.head()

In [ ]:
# Results from fine-tuned whisper base model (lora)
whisper_base_lora_results_filepath = "../outputs/vhp-pre2010-whisper-v2-base-lora-test-sample200-under-30m-gpu/inference_results.parquet"
df_whisper_base_lora_results = pd.read_parquet(whisper_base_lora_results_filepath)
df_whisper_base_lora_results = df_whisper_base_lora_results[df_whisper_base_lora_results['status']=="success"]
df_whisper_base_lora_results = df_whisper_base_lora_results.replace('', np.nan).dropna(subset=['hypothesis'])
df_whisper_base_lora_results['model_name'] = 'whisper-v2-base-lora'
print(df_whisper_base_lora_results.info())
df_whisper_base_lora_results.head()

In [ ]:
# df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set, df_parakeet_results)
df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set, df_faster_whisper_large_v3_results)
# df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set_expanded, df_faster_whisper_small_results)
# df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set_expanded, df_faster_whisper_medium_results)
# df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set_expanded, df_faster_whisper_large_v3_results)
df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set_expanded, df_whisper_base_lora_results)
# df_pre2010_test_set_expanded = df_pre2010_test_set_expanded.dropna(subset=['inference_parakeet-tdt-0.6b-v3', 'inference_faster-whisper-small', 'inference_faster-whisper-medium', 'inference_faster-whisper-large-v3', 'inference_whisper-v2-base-lora'])
df_pre2010_test_set_expanded = df_pre2010_test_set_expanded.dropna(subset=['inference_whisper-v2-base-lora', 'inference_faster-whisper-large-v3'])
# df_pre2010_test_set_expanded[['transcript_raw_text_only', 'inference_parakeet-tdt-0.6b-v3', 'inference_faster-whisper-small', 'inference_faster-whisper-medium', 'inference_faster-whisper-large-v3', 'inference_whisper-v2-base-lora']]
df_pre2010_test_set_expanded[['transcript_raw_text_only', 'inference_whisper-v2-base-lora', 'inference_faster-whisper-large-v3']].head()

In [ ]:
df_pre2010_test_set_expanded.to_csv("../outputs/aggregated_results/vhp-pre2010-stt-comparison-open-models.csv", index=False)

aggregate evaluation results and inference results (faster-whisper-v3)

In [ ]:
df_faster_whispser_with_metadata = pd.merge(df_pre2010_test_set, df_faster_whisper_large_v3_results[['file_id', 'blob_path', 'hypothesis', 'model_name', 'processing_time_sec']], left_on='blob_path', right_on='blob_path', how='left')
df_faster_whispser_with_metadata  = df_faster_whispser_with_metadata.dropna(subset=['hypothesis'])
stt_tag = df_faster_whispser_with_metadata['model_name'].iloc[0]
df_faster_whispser_with_metadata = df_faster_whispser_with_metadata.rename(columns={'hypothesis': f'inference_{stt_tag}', 'processing_time_sec': f'processing_time_{stt_tag}'})

In [ ]:
df_faster_whispser_with_metadata

In [ ]:
df_faster_whispser_with_metadata.to_csv(f"../outputs/aggregated_results/vhp-pre2010-distil-large-v3-full-gpu-inference-merged.csv", index=False)

In [ ]:
# load results from whisper large v3 (baseline) v.s. fine-tuned whisper (lora)
whisper_large_v3_results_filepath = "../outputs/vhp-pre2010-whisper-lora-test-sample200-30m-gpu/inference_results.parquet"

df_whisper_large_v3_results = pd.read_parquet(whisper_large_v3_results_filepath)
df_whisper_large_v3_results = df_whisper_large_v3_results[df_whisper_large_v3_results['status']=="success"]
df_whisper_large_v3_results = df_whisper_large_v3_results.replace('', np.nan).dropna(subset=['hypothesis'])
df_whisper_large_v3_results['model_name'] = 'whisper-large-v3'
print(df_whisper_large_v3_results.info())
df_whisper_large_v3_results.head()

In [ ]:
whisper_lora_results_filepath = "../outputs/vhp-pre2010-whisper-baseline-test-sample200-30m-gpu/inference_results.parquet"

df_whisper_lora_results = pd.read_parquet(whisper_lora_results_filepath)
df_whisper_lora_results = df_whisper_lora_results[df_whisper_lora_results['status']=="success"]
df_whisper_lora_results = df_whisper_lora_results.replace('', np.nan).dropna(subset=['hypothesis'])
df_whisper_lora_results['model_name'] = 'whisper-lora'
print(df_whisper_lora_results.info())
df_whisper_lora_results.head() 

In [ ]:
df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set, df_whisper_large_v3_results)
df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set_expanded, df_whisper_lora_results)
df_pre2010_test_set_expanded = df_pre2010_test_set_expanded.dropna(subset=['inference_whisper-large-v3', 'inference_whisper-lora'])
df_pre2010_test_set_expanded[['transcript_raw_text_only', 'inference_whisper-large-v3', 'inference_whisper-lora']].head()

In [ ]:
df_pre2010_test_set_expanded.to_csv("../outputs/aggregated_results/vhp-pre2010-stt-comparison-model-finetuning-lora.csv", index=False)

In [ ]:
# load results from whisper large v3 (baseline) v.s. fine-tuned whisper (lora)
whisper_large_v3_hpf_results_filepath = "../outputs/vhp-pre2010-whisper-lora-test-sample200-hpf-under-30m-gpu/inference_results.parquet"

df_whisper_large_v3_hpf_results = pd.read_parquet(whisper_large_v3_hpf_results_filepath)
df_whisper_large_v3_hpf_results = df_whisper_large_v3_hpf_results[df_whisper_large_v3_hpf_results['status']=="success"]
df_whisper_large_v3_hpf_results = df_whisper_large_v3_hpf_results.replace('', np.nan).dropna(subset=['hypothesis'])
df_whisper_large_v3_hpf_results['model_name'] = 'whisper-large-v3-input-hpf'
print(df_whisper_large_v3_hpf_results.info())
df_whisper_large_v3_hpf_results.head()

In [ ]:
df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set, df_whisper_large_v3_results)
df_pre2010_test_set_expanded = test_set_add_inference_results(df_pre2010_test_set_expanded, df_whisper_large_v3_hpf_results)
df_pre2010_test_set_expanded = df_pre2010_test_set_expanded.dropna(subset=['inference_whisper-large-v3', 'inference_whisper-large-v3-input-hpf'])
df_pre2010_test_set_expanded[['transcript_raw_text_only', 'inference_whisper-large-v3', 'inference_whisper-large-v3-input-hpf']].head()